In [ ]:
# !gdown 1-F1DO6UNkz3ndjSV4kSzu7zcaQJtf5De

: 

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("SQL Revision Notes.pdf")

pages = loader.load()

: 

In [ ]:
len(pages)

: 

In [ ]:
print(pages[0])

: 

In [ ]:
pages[0].metadata

: 

In [ ]:
pages[0].page_content[0:50]

: 

In [ ]:
# Different Document Loader, YOUTUBE

from langchain_community.document_loaders import YoutubeLoader

# video_url = 'https://www.youtube.com/watch?v=LKCVKw9CzFo'
loader = YoutubeLoader(video_id = "LKCVKw9CzFo", add_video_info = True)
data = loader.load()

len(data)

: 

In [ ]:
data[0].page_content

: 

In [ ]:
# WEBPAGE LOADER

from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://blogs.nvidia.com/blog/what-is-retrieval-augmented-generation/")
docs = loader.load()

docs[0].page_content[:500]

: 

In [ ]:
# CharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter

char_text_splitter = CharacterTextSplitter(
    separator = "",
    chunk_size = 10,
    chunk_overlap = 2
)

# char_text_splitter = CharacterTextSplitter(chunk_size=10, chunk_overlap=2, separator=" ")


: 

In [ ]:
text1 = "abcdefghijklmnopqrstuvacwxyz"
print(char_text_splitter.split_text(text1))

: 

In [ ]:
text1 = "abcdefghijkl mnopqrstuvacwxyz"
print(char_text_splitter.split_text(text1))

: 

In [ ]:
char_text_splitter = CharacterTextSplitter(
    separator = " ",
    chunk_size = 10,
    chunk_overlap = 2
)

text2 = "abcdefghijk lmnopqrs tuvwxyz abcdefg"
print(char_text_splitter.split_text(text2))

: 

In [ ]:
# RecursiveCharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 20
)

text1 = "abcdefghijklmnopqrstuvacwxyz"
print(r_splitter.split_text(text1))

: 

In [ ]:
some_text = """What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.
"""

chunk = r_splitter.split_text(some_text)
print("Chunks: ", chunk)
print("Length of chunks: ", len(chunk))

: 

In [ ]:
for chunk in chunk:
  print(chunk)
  print(len(chunk))
  print("-------------------------------------------")


: 

In [ ]:
from dotenv import load_dotenv
load_dotenv()

: 

In [ ]:
## Embedding using OpenAI model
import openai
import config
import os

from langchain_openai import AzureOpenAIEmbeddings

embedding = AzureOpenAIEmbeddings(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = "2023-03-15-preview",
    model="text-embedding-3-small",
    )

: 

In [ ]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

: 

In [ ]:
import numpy as np

: 

In [ ]:
np.dot(embedding1, embedding2)

: 

In [ ]:
# !gdown 1yVbhJWh4L1unDbDT4APOusTXlwic7aE9
# !gdown 1-F1DO6UNkz3ndjSV4kSzu7zcaQJtf5De

: 

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loaders = [
    PyPDFLoader("PA - Consolidated lecture notes.pdf"),
    PyPDFLoader("SQL Revision Notes.pdf"),
]

docs = []
for loader in loaders:
    docs.extend(loader.load())

: 

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

: 

In [ ]:
splits = text_splitter.split_documents(docs)

: 

In [ ]:
len(splits)

: 

In [ ]:
# FAISS, vectordb for saving embeddings

from langchain_community.vectorstores import FAISS

: 

In [ ]:
db = FAISS.from_documents(
    splits,
    embedding
)

: 

In [ ]:
print(db.index.ntotal)

: 

In [ ]:
query = "What is north star metric"
docs = db.similarity_search(query)
print(docs[0].page_content)

: 

In [ ]:
docs_and_scores = db.similarity_search_with_score(query)
docs_and_scores[0]

: 

In [ ]:
query = "What is primary key"
docs_and_scores = db.similarity_search_with_score(query)

docs_and_scores[0]

: 

In [ ]:
query = "list out some engagement metrics"
docs_and_scores = db.similarity_search_with_score(query,k=3)

docs_and_scores[0]

: 

In [ ]:
query = "list out some engagement metrics"
docs_and_scores = db.similarity_search_with_score(query,k=3)

docs_and_scores[0]

: 

In [ ]:
## Saving the created DB

db.save_local("faiss_db")

: 

In [ ]:
## Loading the saved Vectordb

new_db = FAISS.load_local("faiss_db", embedding, allow_dangerous_deserialization=True)

docs = new_db.similarity_search(query)

: 

In [ ]:
## Question Answering

from langchain.chains import RetrievalQA
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = "2023-03-15-preview",
    temperature = 0.2, 
    model = "gpt-4o-mini"
)

: 

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = new_db.as_retriever()
)

: 

In [ ]:
result = qa_chain.run(query)

: 

In [ ]:
print(result)

: 

In [ ]:
## Using Prompt Template

from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer and dont find it in the given context, just say that you don't know , don't try to make up an answer.
{context}
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

: 

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm =llm,
    retriever = db.as_retriever(),
    return_source_documents = True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

: 

In [ ]:
result = qa_chain({
    "query": query
})

: 

In [ ]:
print(result["result"])

: 

In [ ]:
result["source_documents"][0]

: 

In [ ]:
result = qa_chain({"query": "what is backpropogation in neural networks"})
print(result['result'])

: 

: 